# スタサプ RPA

## モジュール読み込み

In [ ]:
# selenium 4
import math
import time
import os
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.wait import TimeoutException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
# import selenium.webdriver.common.devtools.v114 as devtools
from selenium.webdriver.common.by import By
# .env file load
from dotenv import load_dotenv
load_dotenv()

## インスタンスの生成
### 実行

In [ ]:
options = webdriver.FirefoxOptions()
# options.add_argument('--headless')

driver = webdriver.Firefox(service=FirefoxService(
    GeckoDriverManager().install()), options=options)

driver.set_window_position(0, 0)
original_window = driver.current_window_handle


### .env 読み込み

In [ ]:
service_url = os.getenv("LOGIN_URL")
email = os.getenv("EMAIL_ADDRESS")
password = os.getenv("PASSWORD")

In [ ]:
# 環境変数未設定時
if email == None:
    print("Your ID is None.")
    email = input("Please input ID or email:")

if password == None:
    print("Your PASSWORD is None.")
    password = input("Please input password:")


時刻測定用

In [ ]:
start_time = time.perf_counter()

### ログイン

In [ ]:
driver.get(service_url)
username_input_element = driver.find_element(
    by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/div[2]/input')
password_input_element = driver.find_element(
    by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/div[3]/span/input')
login_button = driver.find_element(
    by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/button[2]/span[2]')

username_input_element.clear()
password_input_element.clear()
username_input_element.send_keys(email)
password_input_element.send_keys(password)
email, password = None, None
login_button.click()


### URL を設定

In [ ]:
studysapuri_uri_dict = {
    "homework_active": "https://learn.studysapuri.jp/ja/todos/active",
    "homework_expired": "https://learn.studysapuri.jp/ja/todos/expired",
}


### 課題状況検査

課題の残りの数をカウントする

残りの課題の数を調べる

In [ ]:
def count_task() -> dict:
    global driver
    driver.get(studysapuri_uri_dict["homework_active"])
    WebDriverWait(driver, timeout=10).until(
        lambda d: d.find_elements(
            by=By.CSS_SELECTOR, value="[class*=BasicTemplate__Body]")
    )

    try:
        is_active_homework_empty = WebDriverWait(driver, timeout=4).until(
            lambda d: d.find_elements(
                by=By.CSS_SELECTOR, value="div > ul li > button")
        )
    except TimeoutException as err:
        is_active_homework_empty = []

    # -------------------------------------------------------------------

    driver.get(studysapuri_uri_dict["homework_expired"])
    WebDriverWait(driver, timeout=4).until(
        lambda d: d.find_elements(
            by=By.CSS_SELECTOR, value="[class*=BasicTemplate__Body]")
    )

    try:
        is_expired_homework_empty = WebDriverWait(driver, timeout=4).until(
            lambda d: d.find_elements(
                by=By.CSS_SELECTOR, value="div > ul li > button")
        )
    except TimeoutException as err:
        is_expired_homework_empty = []
    print(
        f"配信中の課題{len(is_active_homework_empty)}",
        f"期限切れ{len(is_expired_homework_empty)}"
    )

    return {
        "active_homework": len(is_active_homework_empty),
        "expired_homework": len(is_expired_homework_empty)
    }


pass


最初の課題を開く処理

In [ ]:
def first_taskwork_open():
    timeout_sec = 20
    WebDriverWait(driver, timeout=timeout_sec).until(
        lambda d: d.find_element(
            by=By.CSS_SELECTOR, value='button[class*="TodoCard"]'
        )
    )
    first_taskwork = driver.find_element(
        by=By.CSS_SELECTOR, value='button[class*="TodoCard"]')
    first_taskwork.click()

    WebDriverWait(driver, timeout=100).until(
        expected_conditions
        .presence_of_element_located((By.CSS_SELECTOR, 'span[class*=isIncomplete]'))
    )
    # WebDriverWait(driver, timeout=timeout_sec).until(
    #     lambda d: d.find_element(
    #         by=By.CSS_SELECTOR, value='span[class*=isIncomplete]'
    #     )
    # )
    first_todo = driver.find_element(
        by=By.CSS_SELECTOR, value='span[class*=isIncomplete]')
    f = open("JS/incompleteClick.js","r")
    driver.execute_script(f.read())
    f.close()
    # first_todo.click()

開かれた課題を実行する処理

In [ ]:
call_count = 0


def process_todo():
    global driver
    tab = []

    def sub_tab_all_close() -> None:
        for t in tab:
            driver.switch_to.window(t)
            driver.close()

    def quession_automation() -> None:
        print("Call quession_automation")
        """確認問題をランダムに解く
        """
        global call_count
        call_count += 1

        # 待機
        button = WebDriverWait(driver, timeout=10).until(
            lambda d: d.find_element(
                by=By.CSS_SELECTOR, value="[class*=RaisedButton]")
        )
        if not driver.current_url.startswith("https://learn.studysapuri.jp/"):
            driver.close()
            return
        

        print("Call quession_automation wait element,load and body")
        # 完全ロード待ち
        WebDriverWait(driver, timeout=20).until(
            expected_conditions.presence_of_all_elements_located)
        while True:

            # 要素待ち
            WebDriverWait(driver, timeout=20).until(
                lambda d: d.find_element(
                    by=By.CSS_SELECTOR, value="[class*=TopicsPage__Main]  button")
            )
            f = open("JS/autoClick.js","r",encoding='UTF-8')
            js_file_autoClick = f.read()
            driver.execute_script(js_file_autoClick)
            f.close()
            if driver.current_url.endswith("result"):
                # 本当に終了したか判定
                checke_target_button = driver.find_elements(
                    by=By.CSS_SELECTOR, value="button[class*=RaisedButton]")
                if checke_target_button == 0:
                    break
                if checke_target_button == 0 and checke_target_button[0].text != "次の問題へ":
                    break
            time.sleep(0.05)

    # -----------------------------------------------------------------------------------------------
    # 各チャプタのタブを開く
    WebDriverWait(driver, timeout=10).until(lambda d: d.find_element(
        by=By.CSS_SELECTOR, value='ul[class*="LessonStepList"] > li > a')
    )

    after_second_lesston_list = map(lambda element: element.get_attribute("href"), driver.find_elements(
        by=By.CSS_SELECTOR, value='ul[class*="LessonStepList"] > li > a'))

    try:
        for href in list(after_second_lesston_list):
            print(tab,href)
            driver.switch_to.new_window("tab")
            tab.append(driver.current_window_handle)
            driver.get(href)
            if "questions" in href.split("/"):
                print(href)
                quession_automation()

            # video wait
            WebDriverWait(driver, timeout=4).until(
                lambda d: d.find_element(
                    by=By.TAG_NAME, value="video")
            )
            WebDriverWait(driver, timeout=4).until(
                lambda d: d.find_element(
                    by=By.CSS_SELECTOR, value="[class*=hugereplaybutton]")
            )
            f = open("JS/videoAutomation.js","r",encoding="UTF-8")
            command_list = f.readlines()
            for comand in command_list:
                driver.execute_script(comand)
            f.close()
            time.sleep(1)

    except TimeoutException as err:
        print("Timeout Err")

    for i in range(1, 30+1):
        print(i, end="\t")
        time.sleep(1)
    print(tab)
    sub_tab_all_close()
    driver.switch_to.window(original_window)
# -------------------------------------------

driver.switch_to.window(original_window)

In [ ]:
todo_homework = count_task()
while True:
    if todo_homework["active_homework"] != 0: # 配信中の宿題の数が0でないとき
        print("Processing= active_homework")
        driver.get(studysapuri_uri_dict["homework_active"]) # ページ移動
        first_taskwork_open()
        process_todo()
    elif todo_homework["expired_homework"] != 0: # 期限切れのタスクを実行する
        print("Processing= expired_homework")
        driver.get(studysapuri_uri_dict["homework_expired"]) # ページ移動
        first_taskwork_open()
        process_todo()
    else:
        driver.switch_to.window(original_window)
        break
    todo_homework = count_task()

```python

count_task()  # タスクの数を調べる
first_taskwork_open()  # 宿題のページ遷移
process_todo()  # 宿題を始める

```

In [ ]:
end_time = time.perf_counter()
print(f"{end_time-start_time} sec")

driver.quit()  # 終了fileIsolated
